In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print(tf.__version__)

Found GPU at: /device:GPU:0
1.15.2


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
%cd '/content/gdrive/My Drive/tensorflow1/'

/content/gdrive/My Drive/tensorflow1


In [5]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-tk is already the newest version (2.7.17-1~18.04).
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 11 not upgraded.
Need to get 1,791 kB of archives.
After this operation, 7,807 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python-bs4 all 4.6.0-1 [67.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd

In [6]:
%cd /content/gdrive/My Drive/tensorflow1/models/research/
!protoc object_detection/protos/*.proto --python_out=.

/content/gdrive/My Drive/tensorflow1/models/research


In [7]:
import os
os.environ['PYTHONPATH'] += ':/content/gdrive/My Drive/tensorflow1/models/research/:/content/gdrive/My Drive/tensorflow1/models/research/slim' 

In [8]:
import time, psutil
Start = time.time()- psutil.boot_time()
Left= 12*3600 - Start
print('Time remaining for this session is: ', Left/3600)

Time remaining for this session is:  11.923414369093047


In [9]:
pip install tf_slim

     |████████████████████████████████| 358kB 2.7MB/s 


In [10]:
#rember the last CD you did in order to specify the directory.
%cd /content/gdrive/My Drive/tensorflow1/models/research/object_detection/builders/
!python model_builder_test.py

/content/gdrive/My Drive/tensorflow1/models/research/object_detection/builders


In [11]:
%cd /content/gdrive/My Drive/tensorflow1/models/research/object_detection/

/content/gdrive/My Drive/tensorflow1/models/research/object_detection


In [12]:
pip install imageai

     |████████████████████████████████| 184kB 2.8MB/s 


In [13]:
# import the necessary packages
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
import argparse
import imutils
import cv2
import cv2
import os
import re
# import numpy
import os
import time
import os
import dlib
# import numpy as np
from skimage import io
import cv2
from io import open
import requests
import shutil
from zipfile import ZipFile
from imageai.Prediction.Custom import ModelTraining, CustomImagePrediction
import os
import operator


Using TensorFlow backend.


In [14]:
class VideoTransformer():

    def __init__(self, fps=30):
        self.fps = fps

    def video_to_frame(self, path_video, dir_frames):

        vidcap = cv2.VideoCapture(path_video)
        success,image = vidcap.read()
        count = 0
        while success:
            # cv2.imwrite(dir_frames + "/frame_" + str(count).zfill(10) + ".jpg", image)     # save frame as JPEG file
            cv2.imwrite(dir_frames + "/frame_" + str(count) + ".jpg", image)     # save frame as JPEG file      
            success,image = vidcap.read()
            #print('Read a new frame: ', success)
            count += 1


    def frame_to_video(self, dir_frames, path_video, fps):
        num_img = []
        images = []
        for img in os.listdir(dir_frames):
            if img.endswith(".png") or img.endswith(".jpg"):
                # using List comprehension + isdigit() +split() 
                # getting numbers from string  
                data = img.split("_")
                data = data[1].split(".")
                num = data[0]
                #print(num)
                num_img.append(num)
                images.append(img)

        sorted_numbers = numpy.argsort(num_img)
        images_sorted = []
        #print(images)
        for i in range(len(images)):
            images_sorted.append(images[sorted_numbers[i]])

        frame = cv2.imread(os.path.join(dir_frames, images_sorted[0]))
        height, width, layers = frame.shape
        fourrc = cv2.VideoWriter_fourcc(*'MP4V')
        video = cv2.VideoWriter(path_video, fourrc, fps, (width, height))

        for image in images_sorted:
            frame = cv2.imread(os.path.join(dir_frames, image))
            video.write(frame)

        cv2.destroyAllWindows()
        video.release()

In [15]:
# /content/gdrive/My Drive/tensorflow1/models/research/object_detection/test_videos/sample3_720p.mp4

In [38]:
# Set parameters
# /content/gdrive/My Drive/tensorflow1/models/research/object_detection/test_videos/eating.mp4
PATH_VID = "test_videos2/clapping2.mp4"# location video 
DIR_FRAMES = "frames"# directory to store frames (will be created)
PATH_VID_AN = "output/kid.mp4"# name of annotated video
DIR_FRAMES_AN = "labeled_frames/"# directory to store annotated frames (will be created)
VID_FPS = 30
# VID_FPS = 24

print("running")

# Create objects
# img_annotator = ImageAnnotator(PATH_CRED)
video_transf = VideoTransformer(VID_FPS)

# create folder for frames
try:
    os.mkdir(DIR_FRAMES)
except:
    print("Folder " + DIR_FRAMES + " already exists")
    for filename in os.listdir(DIR_FRAMES):
        file_path = os.path.join(DIR_FRAMES, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

try:
    os.mkdir(DIR_FRAMES_AN)
except:
    print("Folder " + DIR_FRAMES_AN + " already exists")
    for filename in os.listdir(DIR_FRAMES_AN):
        file_path = os.path.join(DIR_FRAMES_AN, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

# Turn video into frames
print("Turn video into frames")
video_transf.video_to_frame(PATH_VID, DIR_FRAMES)
print("Done")


running
Folder frames already exists
Folder labeled_frames/ already exists
Turn video into frames
Done


In [39]:
# DIR = '/tmp'
print (len([name for name in os.listdir(DIR_FRAMES) if os.path.isfile(os.path.join(DIR_FRAMES, name))]))
frames_length = len([name for name in os.listdir(DIR_FRAMES) if os.path.isfile(os.path.join(DIR_FRAMES, name))])

265


In [40]:
# os.listdir(DIR_FRAMES)

In [41]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util



### Model preparation variable
MODEL_NAME = 'inference_graph'
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = 'training/labelmap.pbtxt'
NUM_CLASSES = 15 #remember number of objects you are training? cool.


### Load a (frozen) Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')


###Loading label map
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)




### Load image into numpy function
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)




###STATING THE PATH TO IMAGES TO BE TESTED
PATH_TO_TEST_IMAGES_DIR = 'frames/'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'frame_{}.jpg'.format(i)) for i in range(1, frames_length) ]
IMAGE_SIZE = (12, 8)




### Function to run inference on a single image which will later be used in an iteration
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[1], image.shape[2])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: image})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.int64)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict


In [42]:
### To iterate on each image in the test image path defined 
### NB define the range of numbers and let it match the number of imAGES IN TEST FOLDER +1
image_no = 1
for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  image_name = image_path.split('/')[-1].strip('.jpg')
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=1,
      min_score_thresh=0.90)
  # display(Image.fromarray(image_np))
  image1 = Image.fromarray(image_np)
  path_image= "/content/gdrive/My Drive/tensorflow1/models/research/object_detection/labeled_frames/" + str(image_name) + '.jpg'
  print(image_name)
  image1.save(path_image, 'JPEG')
  image_no += 1
  
  # cv2.imwrite(path_img_an, image_np)



frame_1
frame_2
frame_3
frame_4
frame_5
frame_6
frame_7
frame_8
frame_9
frame_10
frame_11
frame_12
frame_13
frame_14
frame_15
frame_16
frame_17
frame_18
frame_19
frame_20
frame_21
frame_22
frame_23
frame_24
frame_25
frame_26
frame_27
frame_28
frame_29
frame_30
frame_31
frame_32
frame_33
frame_34
frame_35
frame_36
frame_37
frame_38
frame_39
frame_40
frame_41
frame_42
frame_43
frame_44
frame_45
frame_46
frame_47
frame_48
frame_49
frame_50
frame_51
frame_52
frame_53
frame_54
frame_55
frame_56
frame_57
frame_58
frame_59
frame_60
frame_61
frame_62
frame_63
frame_64
frame_65
frame_66
frame_67
frame_68
frame_69
frame_70
frame_71
frame_72
frame_73
frame_74
frame_75
frame_76
frame_77
frame_78
frame_79
frame_80
frame_81
frame_82
frame_83
frame_84
frame_85
frame_86
frame_87
frame_88
frame_89
frame_90
frame_91
frame_92
frame_93
frame_94
frame_95
frame_96
frame_97
frame_98
frame_99
frame_100
frame_101
frame_102
frame_103
frame_104
frame_105
frame_106
frame_107
frame_108
frame_109
frame_110
frame_11

In [14]:
.
%cd /content/gdrive/My Drive/tensorflow1/models/research/object_detection/

SyntaxError: ignored

In [15]:
# import numpy
# t1 = time.perf_counter()

# # iterate through frames
# print("Get informations from images")
# # for file in os.listdir(DIR_FRAMES):
# #     if file.endswith(".jpg") or file.endswith(".png"):
# #         # Get information from images
# #         img_objects = img_annotator.get_objects(DIR_FRAMES+file, True)

# #         # Annotate images
# #         img_annotator.annotate_image(DIR_FRAMES+file, DIR_FRAMES_AN+file, img_objects)

# # transform frames to video
# print("Transform frames to video")
# video_transf.frame_to_video(DIR_FRAMES_AN, PATH_VID_AN,VID_FPS)

# t2 = time.perf_counter()
# print(f'Finished in {t2-t1} seconds')


# print("Done!")

In [17]:
import cv2
import numpy as np
import os
 
from os.path import isfile, join
 
def convert_frames_to_video(pathIn,pathOut,fps):
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
 
    #for sorting the file names properly
    # files.sort(key = lambda x: int(x[5:-4]))
 
    for i in range(1,len(files)):
        filename=pathIn + 'frame_'+str(i)+'.jpg'
        #reading each files
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        print(filename)
        #inserting the frames into an image array
        frame_array.append(img)
 
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
 
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()
 
def main():
    pathIn= '/content/gdrive/My Drive/tensorflow1/models/research/object_detection/labeled_frames/'
    pathOut = '/content/gdrive/My Drive/tensorflow1/models/research/object_detection/video_results/4.avi'
    fps = 25.0
    convert_frames_to_video(pathIn, pathOut, fps)
 
 
if __name__=="__main__":
    main()

/content/gdrive/My Drive/tensorflow1/models/research/object_detection/labeled_frames/frame_1.jpg
/content/gdrive/My Drive/tensorflow1/models/research/object_detection/labeled_frames/frame_2.jpg
/content/gdrive/My Drive/tensorflow1/models/research/object_detection/labeled_frames/frame_3.jpg
/content/gdrive/My Drive/tensorflow1/models/research/object_detection/labeled_frames/frame_4.jpg
/content/gdrive/My Drive/tensorflow1/models/research/object_detection/labeled_frames/frame_5.jpg
/content/gdrive/My Drive/tensorflow1/models/research/object_detection/labeled_frames/frame_6.jpg
/content/gdrive/My Drive/tensorflow1/models/research/object_detection/labeled_frames/frame_7.jpg
/content/gdrive/My Drive/tensorflow1/models/research/object_detection/labeled_frames/frame_8.jpg
/content/gdrive/My Drive/tensorflow1/models/research/object_detection/labeled_frames/frame_9.jpg
/content/gdrive/My Drive/tensorflow1/models/research/object_detection/labeled_frames/frame_10.jpg
/content/gdrive/My Drive/tens

In [ ]:
#  filename = "/content/gdrive/My Drive/tensorflow1/models/research/object_detection/labeled_frames/frame_539.jpg"
#  img = cv2.imread(filename)

In [ ]:
# frame_array = []
# files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
# pathIn= '/content/gdrive/My Drive/tensorflow1/models/research/object_detection/labeled_frames/'
# for i in range(len(files)):
#         filename=pathIn + 'frame_'+str(i)+'.jpg'
#         print(filename)

/content/gdrive/My Drive/tensorflow1/models/research/object_detection/labeled_frames/frame_0.jpg
/content/gdrive/My Drive/tensorflow1/models/research/object_detection/labeled_frames/frame_1.jpg
/content/gdrive/My Drive/tensorflow1/models/research/object_detection/labeled_frames/frame_2.jpg
/content/gdrive/My Drive/tensorflow1/models/research/object_detection/labeled_frames/frame_3.jpg
/content/gdrive/My Drive/tensorflow1/models/research/object_detection/labeled_frames/frame_4.jpg
/content/gdrive/My Drive/tensorflow1/models/research/object_detection/labeled_frames/frame_5.jpg
/content/gdrive/My Drive/tensorflow1/models/research/object_detection/labeled_frames/frame_6.jpg
/content/gdrive/My Drive/tensorflow1/models/research/object_detection/labeled_frames/frame_7.jpg
/content/gdrive/My Drive/tensorflow1/models/research/object_detection/labeled_frames/frame_8.jpg
/content/gdrive/My Drive/tensorflow1/models/research/object_detection/labeled_frames/frame_9.jpg
/content/gdrive/My Drive/tenso

In [ ]:
# Image_detection

In [ ]:
#video_detection